In [1]:
import os
import glob
import argparse
import pandas as pd
from multiprocessing import Pool

# split into 25 files
parser = argparse.ArgumentParser()
parser.add_argument("--dataDir", "-d", help="The run directory")
parser.add_argument("--prefix", "-p", help="The prefix of input fastq file")
parser.add_argument("--thread", "-t", help="Thread number to run")
args = parser.parse_args()
data_dir = args.dataDir
prefix = args.prefix
file_number = int(args.thread)
###################################################################
# data_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/GATK_best_practices/heart_3"
# prefix = "heart_3_L6-"
# file_number = 25
############################################################
in_file="%s/%srmhex.rmsk.txt" % (data_dir, prefix)
result_dir = os.path.join(os.path.dirname(in_file), "filter_intron_near_splicejuncts")
result_file = os.path.join(os.path.dirname(in_file), "%srmhex.rmsk.rmintron.txt" % prefix)
snpir_dir = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/SNPiR-master/"
##############################################################################################################
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
os.chdir(result_dir)
total_lines = int(os.popen("wc -l %s" % in_file).read().split()[0])
line_number = int(total_lines / file_number) + 1
os.system("split -l %d %s %s" % (line_number, in_file, prefix))

# filter_mismatch_first6bp
def filter_intron_near_splicejuncts(in_file):
    os.chdir(snpir_dir)
    annotation_file = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/SNPiR-master/data/GENCODE_v32_modified_sorted.txt"
    os.system("./filter_intron_near_splicejuncts.pl -infile %s -outfile %s/result_%s -genefile %s" % (in_file, result_dir, os.path.basename(in_file), annotation_file))
    
in_list = glob.glob("%s/%s*" % (result_dir, prefix))
pool = Pool(processes=file_number)
for x in in_list:
    pool.apply_async(filter_intron_near_splicejuncts, (x, ))
pool.close()
pool.join()

# merge result
i_list = [x for x in glob.glob("%s/result_*" % result_dir) if "_tmp" not in x if "_failed" not in x]
os.system("cat %s | sort -k1,1 -k2,2n > %s" % (" ".join(i_list), result_file))

0